In [1]:
import os
import json
import requests
import pandas as pd
import time

In [2]:
google_place = os.environ["GOOGLE_API"]

In [73]:
# Google Places endpoint used
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# Some brief searching found that Vancouver's SkyTrain stations are categorized as subway_station on Google maps
params = {
    'query': 'Vancouver SkyTrain Stations',
    'radius': 40000,  # Radius in meters
    'type': 'subway_station',
    'key': google_place
}

# Send the initial request
response = requests.get(url, params=params)

# Parse the response as JSON
data = response.json()

# Initialize an empty list to store SkyTrain station details
skytrain_details = []

# Handle the response data
if response.status_code == 200:
    results = data['results']
    
    for result in results:
        name = result['name']
        rating = result.get('rating', 'N/A')
        rating_count = result['user_ratings_total'] 
        place_id = result['place_id']
        location = result['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']

        skytrain_details.append({
            'Name': name,
            'Rating': rating,
            'Number of Ratings': rating_count,
            'Place ID': place_id,
            'Latitude': latitude,
            'Longitude': longitude
        })

    # Check if there are more results and retrieve them using pagination
    while 'next_page_token' in data:
        next_page_token = data['next_page_token']
        params['pagetoken'] = next_page_token
        
        # allow the next page token to become valid
        time.sleep(4)
        
        # Send the next page request
        response = requests.get(url, params=params)
        
        # Parse the response as JSON
        data = response.json()
        
        if response.status_code == 200:
            results = data['results']
            
            for result in results:
                name = result['name']
                rating = result.get('rating', 'N/A')
                rating_count = result['user_ratings_total'] 
                place_id = result['place_id']
                location = result['geometry']['location']
                latitude = location['lat']
                longitude = location['lng']
                
                skytrain_details.append({
                    'Name': name,
                    'Rating': rating,
                    'Number of Ratings': rating_count,
                    'Place ID': place_id,
                    'Latitude': latitude,
                    'Longitude': longitude
                })
        else:
            print('Request failed with status code:', response.status_code)
else:
    print('Request failed with status code:', response.status_code)

# Create a DataFrame from the skytrain details list
SkyTrainDF = pd.DataFrame(skytrain_details)
# Store the DataFrame as a csv file
SkyTrainDF.to_csv('skytrain_stations.csv', index=False)

In [74]:
SkyTrainDF

,Name,Rating,Number of Ratings,Place ID,Latitude,Longitude
0,Broadway-City Hall Station,4.1,90,ChIJQx4neN1zhlQRdHFbwLLpxeM,49.262997,-123.114541
1,Vancouver City Centre,4.3,146,ChIJ04sXE39xhlQR1Woi1gmeQXY,49.282469,-123.118613
2,King Edward Station @ Platform 1,4.0,36,ChIJifoZXPJzhlQRtHmyjDhlNAc,49.249187,-123.115857
3,Marine Drive Station,4.2,91,ChIJ9_pZvfZ0hlQRtF0YmDWM4lI,49.209670,-123.117020
4,Granville,4.0,89,ChIJFxdoy3hxhlQRXYS-3DIa4Sk,49.283292,-123.116134
5,Olympic Village Station,4.1,54,ChIJlfAnGNxzhlQR-C1WKlNCF0g,49.266540,-123.115620
6,Burrard,4.2,181,ChIJWWb7iIFxhlQRBL4QZPdZs4I,49.285553,-123.120204
7,Sea Island Centre,3.5,13,ChIJfdGuxM10hlQRc5O28XA1Muk,49.193046,-123.157890
8,Stadium-Chinatown,3.6,158,ChIJOd5wkHtxhlQRZ-hsRW79874,49.279642,-123.109815
9,Templeton,4.1,38,ChIJJ7z-3MR0hlQRdu2zx6WWbTc,49.196481,-123.146402
